In [1]:
import sys
sys.path.append('../')
# from experiments import utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plot.utils import bar_update_fig, line_update_fig
import pypolo2

width = 1000
height = 800
markersize = 10
linewidth = 4
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
linetype = {'Low':'dot','Medium':'dashdot','High':'longdash'}
# 创建一个包含线型的示例数组
line_dash = ['solid', 'dot', 'dash', 'dashdot', 'longdash']
colors_dict = ['#C74223', '#3D9182', '#245C8F']

# average suppression of pm2.5 concentration for different teamsize

In [ ]:
# 累计pm2.5控制总量
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "EffectOrientedGreedySpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_pm25suppression = []
sche_step = 0
adaptive_step = 0
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25suppression = []
    for i in range(3):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        seed_pm25suppression = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            effect = np.sum(data['spray_effect'])/48/400
            # if effect < 10:
            #     effect = 0
            seed_pm25suppression.append(effect)
        variable_pm25suppression.append(np.mean(seed_pm25suppression))
    strategy_pm25suppression.append(variable_pm25suppression)
strategy_pm25suppression = np.array(strategy_pm25suppression)

width = 1100
height = 750
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(['3 Sprinkle Truck', '5 Sprinkle Truck', '7 Sprinkle Truck'])
xtitle = 'Sprinkle Truck Number'
ytitle = 'cumulative suppression of pm2.5(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(
    width= 0.2, 
    x=labels, 
    y=strategy_pm25suppression[0,:], 
    name='SmartSpr', 
    marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(
    width= 0.2, 
    x=labels, 
    y=strategy_pm25suppression[1,:], 
    name='EffectOrientedGreedySprinkle', 
    marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(
    width= 0.2, 
    x=labels, 
    y=strategy_pm25suppression[2,:], 
    name='MaximumCoverageSprinkle', 
    marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(
    width= 0.2, 
    x=labels, 
    y=strategy_pm25suppression[3,:], 
    name='Non-Sprinkle', 
    marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25suppression)+0.3]),  # 设置y轴范围
    legend=dict(font=dict(size=25), x=0.5, y=1.12, orientation='h')
    )
fig.show()
filename = './outputs/png/bar_AverageSprayEffect-teamsize357-Greedy.png'
print(filename)
fig.write_image(filename, scale=1)

In [ ]:
width = 1050
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(['3 Sprinkle Truck', '5 Sprinkle Truck', '7 Sprinkle Truck'])
xtitle = 'Sprinkle Truck Number'
ytitle = 'Average suppression of pm2.5(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[0,:], name='SmartSpr', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
# fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25suppression[1,:], name='EffectOrientedGreedySpray', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[2,:], name='MaximumCoverageSprinkle', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[3,:], name='Non-Sprinkle', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25suppression)+0.3]),
    legend=dict(font=dict(size=32), x=0.5, y=1.12, orientation='h')
    ) # name size
fig.show()
filename = './outputs/png/bar_AverageSprayEffect-teamsize357.png'
print(filename)
fig.write_image(filename, scale=1)

# average pm2.5 concentration at last two hours

In [ ]:
#区域平均污染物浓度
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "EffectOrientedGreedySpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25concentration = []
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25concentration = []
    for i in range(3):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        seed_pm25concentration = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            time_pm25concentration = []
            for j in range(total_step):
                if j > 24:
                    pm25concentration = np.mean(data['truth_env'][j])
                    time_pm25concentration.append(pm25concentration)
            seed_pm25concentration.append(np.mean(time_pm25concentration))
        variable_pm25concentration.append(np.mean(seed_pm25concentration))
    strategy_pm25concentration.append(variable_pm25concentration)
strategy_pm25concentration = np.array(strategy_pm25concentration)
strategy_pm25concentration[-1,:] = np.mean(strategy_pm25concentration[-1,:])

width = 1000
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(['3 Sprinkle Truck', '5 Sprinkle Truck', '7 Sprinkle Truck'])
xtitle = 'Sprinkle Truck Number'
ytitle = 'pm2.5 concentration(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25concentration[0,:], name='SmartSpr', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25concentration[1,:], name='EffectOrientedGreedySprinkle', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25concentration[2,:], name='MaximumCoverageSprinkle', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25concentration[3,:], name='Non-Sprinkle', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25concentration)+5]),
    legend=dict(font=dict(size=20), x=0.5, y=1.12, orientation='h')
    ) # name size
fig.show()
filename = './outputs/png/bar_AveragePollution-teamsize357-Greedy.png'
print(filename)
fig.write_image(filename, scale=1)

In [ ]:
width = 1050
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(['3 Sprinkle Truck', '5 Sprinkle Truck', '7 Sprinkle Truck'])
xtitle = 'Sprinkle Truck Number'
ytitle = 'pm2.5 concentration(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[0,:], name='SmartSpr', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
# fig.add_trace(go.Bar(width= 0.2, x=labels, y=strategy_pm25concentration[1,:], name='EffectOrientedGreedySpray', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[2,:], name='MaximumCoverageSprinkle', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[3,:], name='Non-Sprinkle', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25concentration)+5]),
    legend=dict(font=dict(size=32), x=0.5, y=1.12, orientation='h')
    ) # name size
fig.show()
filename = './outputs/png/bar_AveragePollution-teamsize357.png'
print(filename)
fig.write_image(filename, scale=1)

# average suppression of pm2.5 concentration for different source number

In [3]:
# 累计pm2.5控制总量
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_6","teamsize_9"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_pm25suppression = []
sche_step = 0
adaptive_step = 0
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25suppression = []
    for i in range(3):
        comparing_variable = comparing_variables[1][i]
        team_size = 5
        sourcenum = sourcenums[i]
        seed_pm25suppression = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            effect = np.sum(data['spray_effect'])/48/400
            seed_pm25suppression.append(effect)
        variable_pm25suppression.append(np.mean(seed_pm25suppression))
    strategy_pm25suppression.append(variable_pm25suppression)
strategy_pm25suppression = np.array(strategy_pm25suppression)

width = 1050
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(["Numsource 2","Numsource 4","Numsource 6"])
xtitle = 'Pollution Source Number'
ytitle = 'Average suppression of pm2.5(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(
    width= 0.23, 
    x=labels, 
    y=strategy_pm25suppression[0,:], 
    name='SmartSpr', 
    marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(
    width= 0.23, 
    x=labels, 
    y=strategy_pm25suppression[1,:], 
    name='MaximumCoverageSprinkle', 
    marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(
    width= 0.23, 
    x=labels, 
    y=strategy_pm25suppression[2,:], 
    name='Non-Sprinkle', 
    marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25suppression)+0.3]),  # 设置y轴范围
    legend=dict(font=dict(size=32), x=0.5, y=1.12, orientation='h')
    )
fig.show()
filename = './outputs/png/bar_AverageSprayEffect-numsource2468.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/bar_AverageSprayEffect-numsource2468.png


# average pm2.5 concentration at last two hours for different source number

In [4]:
#区域平均污染物浓度
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48
sche_step = 0
adaptive_step = 0
strategy_pm25concentration = []
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_pm25concentration = []
    for i in range(3):
        comparing_variable = comparing_variables[1][i]
        team_size = 5
        sourcenum = sourcenums[i]
        seed_pm25concentration = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            time_pm25concentration = []
            for j in range(total_step):
                if j > 24:
                    pm25concentration = np.mean(data['truth_env'][j])
                    time_pm25concentration.append(pm25concentration)
            seed_pm25concentration.append(np.mean(time_pm25concentration))
        variable_pm25concentration.append(np.mean(seed_pm25concentration))
    strategy_pm25concentration.append(variable_pm25concentration)
strategy_pm25concentration = np.array(strategy_pm25concentration)

width = 1050
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(["Numsource 2","Numsource 4","Numsource 6"])
xtitle = 'Pollution Source Number'
ytitle = 'pm2.5 concentration(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[0,:], name='SmartSpr', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[1,:], name='MaximumCoverageSprinkle', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[2,:], name='Non-Sprinkle', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=32),tickfont=dict(size=32))
fig.update_layout(
    yaxis=dict(range=[0, np.max(strategy_pm25concentration)+5]),
    legend=dict(font=dict(size=32), x=0.5, y=1.12, orientation='h')
    ) # name size
fig.show()
filename = './outputs/png/bar_AveragePollution-numsource2468.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/bar_AveragePollution-numsource2468.png


# cumulative suppression of pm2.5 concentration for different R change intervals

In [ ]:
# 累计pm2.5控制总量
Setting = pypolo2.utilities.Config()
strategy_name = ["PINNsOnlySpray",
                 "SequentialSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_6","teamsize_9"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,20,32,42,50,55,60,66,70]
team_sizes = [3,6,9]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_pm25suppression = []
for strategy in strategy_name:
    variable_pm25suppression = []
    for i in range(3):
        comparing_variable = comparing_variables[2][i]
        team_size = 5
        sourcenum = 4
        seed_pm25suppression = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{Setting.sche_step}_AS{Setting.adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            effect = np.sum(data['spray_effect'])
            seed_pm25suppression.append(effect)
        variable_pm25suppression.append(np.mean(seed_pm25suppression))
    strategy_pm25suppression.append(variable_pm25suppression)
strategy_pm25suppression = np.array(strategy_pm25suppression)

width = 1000
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(["change_interval4","change_interval8","change_interval12"])
xtitle = 'source change interval'
ytitle = 'cumulative suppression of pm2.5(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[0,:], name='EffectOrientedSpray', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[1,:], name='MaximumCoverageSpray', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25suppression[2,:], name='NoSpray', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_layout(legend=dict(font=dict(size=26), x=0.5, y=1.12, orientation='h')) # name size
fig.show()
# filename = '../outputs/png/bar_原污染源消失产生新污染源.png'
# print(filename)
# fig.write_image(filename, scale=1)

# average pm2.5 concentration at last two hours for different source change interval

In [ ]:
#区域平均污染物浓度
Setting = pypolo2.utilities.Config()
strategy_name = ["PINNsOnlySpray",
                 "SequentialSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_6","teamsize_9"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,20,32,42,50,55,60,66,70]
team_sizes = [3,6,9]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
total_step = 48

strategy_pm25concentration = []
for strategy in strategy_name:
    variable_pm25concentration = []
    for i in range(3):
        comparing_variable = comparing_variables[2][i]
        team_size = 5
        sourcenum = 4
        seed_pm25concentration = []
        for seed in seeds:
            pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{Setting.sche_step}_AS{Setting.adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
            pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
            data = pypolo2.experiments.utilities.readpkl(pkldir)
            time_pm25concentration = []
            for j in range(total_step):
                if j > 36:
                    pm25concentration = np.mean(data['truth_env'][j])
                    time_pm25concentration.append(pm25concentration)
            seed_pm25concentration.append(np.mean(time_pm25concentration))
        variable_pm25concentration.append(np.mean(seed_pm25concentration))
    strategy_pm25concentration.append(variable_pm25concentration)
strategy_pm25concentration = np.array(strategy_pm25concentration)

width = 1000
height = 650
colors_dict = ['#C74223', '#3D9182', '#245C8F']

labels = np.array(["change_interval4","change_interval8","change_interval12"])
xtitle = 'source change interval'
ytitle = 'pm2.5 concentration(μg/m³)'
fig = go.Figure()
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[0,:], name='EffectOrientedSpray', marker=dict(line_color=colors_dict[2],color=colors_dict[2], pattern_shape='\\')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[1,:], name='MaximumCoverageSpray', marker=dict(line_color=colors_dict[0],color=colors_dict[0], pattern_shape='x')))
fig.add_trace(go.Bar(width= 0.23, x=labels, y=strategy_pm25concentration[2,:], name='NoSpray', marker=dict(line_color=colors_dict[1],color=colors_dict[1], pattern_shape='+')))

fig.update_traces(
    marker=dict(pattern_fillmode="replace", line_width=3, pattern_size=15),
)

fig = bar_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_layout(legend=dict(font=dict(size=26), x=0.5, y=1.12, orientation='h')) # name size
fig.show()
# filename = '../outputs/png/bar_原污染源消失产生新污染源.png'
# print(filename)
# fig.write_image(filename, scale=1)